In [1]:
using JLD2
using FileIO
using PyPlot
using Statistics
using StatsBase 
using Distributions
using LinearAlgebra
using HDF5
using IntervalSets
using Random, ArraysOfArrays
using ValueShapes
using Measurements
using BAT
using DSP

In [2]:
FPATH = "../../data/sampling_results/Benchmark-5/"

function hist_2d(name, dim_1, dim_2; nbins=40, axmin = true, axmax=true)
    smpl_tmp = bat_read(FPATH*name).result
    smpl_tmp = varshape(prior).(smpl_tmp)
    smpl_tmp, smpl_w = flatview(unshaped.(smpl_tmp.v)), smpl_tmp.weight
    if axmin == true
        hist_tmp = fit(Histogram, (smpl_tmp[dim_1,:],smpl_tmp[dim_2,:]), FrequencyWeights(smpl_w), nbins=nbins)
    else
       hist_tmp = fit(Histogram, (smpl_tmp[dim_1,:],smpl_tmp[dim_2,:]), FrequencyWeights(smpl_w), (range(axmin[1], stop=axmax[1], length=nbins), range(axmin[2], stop=axmax[2], length=nbins))) 
    end
    hist_tmp = normalize(hist_tmp, mode=:pdf) 
    return hist_tmp
end

function hist_1d(name, dim_1; nbins=40)
    smpl_tmp = bat_read(FPATH*name).result
    smpl_tmp = varshape(prior).(smpl_tmp)
    smpl_tmp, smpl_w = flatview(unshaped.(smpl_tmp.v)), smpl_tmp.weight
    hist_tmp = fit(Histogram, smpl_tmp[dim_1,:], FrequencyWeights(smpl_w), nbins=nbins)
    hist_tmp = normalize(hist_tmp, mode=:pdf)
    return hist_tmp
end

function marg_mean(name, dim_1, dim_2)
    smpl_tmp = bat_read(FPATH*name).result
    smpl_tmp = varshape(prior).(smpl_tmp)
    smpl_tmp = unshaped(mode(smpl_tmp))
    return smpl_tmp[dim_1], smpl_tmp[dim_2]
end

function marg_std(name, dim_1, dim_2)
    smpl_tmp = bat_read(FPATH*name).result
    smpl_tmp = varshape(prior).(smpl_tmp)
    smpl_tmp = unshaped(std(smpl_tmp))
    return smpl_tmp[dim_1], smpl_tmp[dim_2]
end

marg_std (generic function with 1 method)

In [3]:
β1 = 0.015
β2 = 0.0077
β3 = 0.0058 

prior = NamedTupleDist(
        tr_size = [truncated(Normal(0.2, 0.04), 0.03, 0.19), truncated(Normal(0.2, 0.04), 0.03, 0.19)],
        tr_size_2 = [truncated(Normal(0.2, 0.04), 0.03, 0.19), truncated(Normal(0.2, 0.04), 0.03, 0.19)],
        ang_spr = [truncated(Normal(4.0, 2.0), 1.0, 8.0), truncated(Normal(4.0, 2.0), 1.0, 8.0)],
        ang_spr_2 = [truncated(Normal(4.0, 2.0), 1.0, 4.0), truncated(Normal(4.0, 2.0), 1.0, 4.0)],
        mixt_pow =  0.35 .. 1.0, 
        waist = [truncated(Normal(2.774, 0.03), 2.5, 3.6)],
        waist_2 = [truncated(Normal(2.774, 0.03), 2.5, 3.6)],
        algmx = [23.0 .. 48, 23.0 .. 48.0, 10.0 .. 30.0, 23.0 .. 48.0],
        algmy = [23.0 .. 48, 23.0 .. 48.0, 10.0 .. 30.0, 23.0 .. 48.0],
        cam4_ped = 4.0 .. 40.0,
        cam4_light_fluct = 1.0 .. 3.0,
        cam4_light_amp = 1.6 .. 9.9, 
        resx = [1, 1, 1], # 23, 24, 25, 
        resy = [1, 1, 1], # 26,27, 28, 
        cam4_resx = truncated(Normal(3, 1.5), 0, Inf),
        cam4_resy = truncated(Normal(3, 1.5), 0, Inf), 
        psx = [27.1, 21.6, 114.0], # 31, 32, 33
        psy = [30.5, 23.4, 125.0], # 34, 35, 36
        cam4_psx = 121.8, # 37
        cam4_psy = 120.0, # 38
        light_amp  = [1.0 .. 13.0 , 1.0 .. 17.0, 1.0 .. 5.0], # 1.0 .. 5.0
        s_cam = [0.0, 1.47799, 15.025999, 23.1644],
); # new prior, waist and positions are corrected


labels = [L"\sigma_x", L"\sigma_y", L"\sigma_x", L"\sigma_y", L"\sigma'_x", L"\sigma'_y", L"\sigma'_x", L"\sigma'_y", L"\alpha", L"s", L"s", 
    L"\mu_x", L"\mu_x", L"\mu_x", L"\mu_x", L"\mu_y", L"\mu_y", L"\mu_y", L"\mu_y", 
    "ped", "lf", "a", 
    "res x", "res x", "res x", "res y", "res y", "res y", "res x", "res y", 
    "ps x", "ps x", "ps x", "ps y", "ps y", "ps y", "ps x", "ps y", 
    "la", "la", "la"
    ];

In [4]:
filenames = readdir("../../data/sampling_results/Benchmark-5/")
images = load("../../data/experiment/dataset_2/m2/images-satur.jld2");
hc_mask = [i[1:3] == "hc-" for i in filenames]
hc_fnames = filenames[hc_mask]
hc_eind = parse.(Int64, [i[4:end-5] for i in hc_fnames])
hc_charge = images["charge"][hc_eind]
hc_runind = images["ev_ind"][hc_eind];

In [5]:
filenames = readdir("../../data/sampling_results/Benchmark-5/")
images = load("../../data/experiment/dataset_2/m1/images-satur.jld2");
lc_mask = [i[1:3] == "lc-" for i in filenames]
lc_fnames = filenames[lc_mask]
lc_eind = parse.(Int64, [i[4:end-5] for i in lc_fnames])
lc_charge = images["charge"][lc_eind]
lc_runind = images["ev_ind"][lc_eind];

In [6]:
fnames = [lc_fnames; hc_fnames]
charges = [lc_charge; hc_charge]
runind = [lc_runind ; hc_runind]
charges_n = charges ./ maximum(charges)
runind_n = runind ./ maximum(runind);

In [7]:
length(fnames)

672

In [8]:
msklc = 1:length(lc_fnames)
mskhc = length(lc_fnames)+1:length(lc_fnames)+length(hc_fnames);

In [9]:
charges_n[sortperm(charges_n)] .= 1:length(charges_n)
charges_n = charges_n ./ maximum(charges_n);

## Plot 1: 

In [ ]:
hists = [hist_2d(i, 1,2) for i in fnames]
hists_2 = [hist_2d(i, 3,4) for i in fnames]
mean_vals = [marg_mean(i, 1,2) for i in fnames]
mean_vals_2 = [marg_mean(i, 3,4) for i in fnames]
std_vals = [marg_std(i, 1,2) for i in fnames]
std_vals_2 = [marg_std(i, 3,4) for i in fnames]
hist1d_1 = [hist_1d(i, 1) for i in fnames]
hist1d_2 = [hist_1d(i, 2) for i in fnames];

In [ ]:
xmin, xmax = 0.02, 0.24
xrange=range(xmin, stop = xmax,length=100)
yrange=range(xmin, stop = xmax,length=100)
xvals = [pdf(truncated(Normal(0.2, 0.04), 0.02, 0.25), x) for x in xrange]
yvals = [pdf(truncated(Normal(0.2, 0.04), 0.02, 0.25), y)  for y in yrange]
z = xvals*yvals';

In [ ]:
hists = [hist_2d(i, 1,2, axmin=[0.02,0.02], axmax=[0.18,0.18], nbins=150,) for i in fnames];
hists_2 = [hist_2d(i, 3,4, axmin=[0.02,0.02], axmax=[0.18,0.18], nbins=150,) for i in fnames];

In [ ]:
fnames[argmax([i[2] for i in std_vals])]

In [ ]:
argmax([i[2] for i in std_vals])

In [ ]:
mean_vals[449][1]

In [ ]:
fig, ax= plt.subplots(2,2, figsize=(7,7), gridspec_kw=Dict("width_ratios"=>[6,1], "height_ratios"=>[1,6]))
fig.subplots_adjust(hspace=0.0, wspace=0.0)

# ax[2,1].contourf(xrange, yrange, z, levels=range(0.1*minimum(z), maximum(z), length=25)[1:end], cmap="bone_r", alpha=0.5, antialiased=true) 
range_tmp = range(0, stop=maximum(z), length=12)[2:end]
colors_tmp = [plt.cm.bone_r(i) for i in range(0, stop=0.4, length=length(range_tmp))[2:end]]
ax[2,1].contourf(xrange, yrange, z, levels=range_tmp, colors=colors_tmp, alpha=0.4, antialiased=true) 

sum_weights = zeros(size(hists[1].weights'))
sum_weights_2 = zeros(size(hists_2[1].weights'))
for i in eachindex(hists)
    sum_weights = sum_weights .+ hists[i].weights'
    sum_weights_2 = sum_weights_2 .+ hists_2[i].weights'
end

label_1 = "Data (Halo)"
label_2 = "Data (Core)"
for i in eachindex(hists)[end:-1:1]  
    ax[2,1].errorbar(mean_vals[i][1], mean_vals[i][2], xerr=std_vals[i][1], yerr=std_vals[i][2], 
        color="k", 
        fmt="o", markersize=4, 
        mfc=plt.cm.OrRd(charges_n[i]), #red
        markeredgecolor = "k", markeredgewidth=0.4,
        capsize=1.5, ecolor="k", elinewidth=0.4, label=label_1)
    label_1 = ""
    ax[2,1].errorbar(mean_vals_2[i][1], mean_vals_2[i][2], xerr=std_vals_2[i][1], yerr=std_vals_2[i][2], 
        color="k", 
        fmt="o", markersize=4,
        mfc=plt.cm.GnBu(charges_n[i]), #red
        markeredgecolor = "k", markeredgewidth=0.4,
        capsize=1.5, ecolor="k", elinewidth=0.4, label=label_2)
    label_2 = ""
end

cb = ax[2,1].scatter(charges_n, charges_n, s=0.0, c=charges, cmap="OrRd")
cb2 = ax[2,1].scatter(charges_n, charges_n, s=0.0, c=charges, cmap="GnBu")
ax_ins_3 = fig.add_axes([0.19,0.55,0.02,0.2])
ax_ins_2 = fig.add_axes([0.15,0.55,0.02,0.2])
fig.colorbar(cb, spacing="proportional", label="Population", cax=ax_ins_3)
fig.colorbar(cb2, spacing="proportional", ticks=[], cax=ax_ins_2)

proxy = plt.Rectangle((10, 10), 1, 1, fc=plt.cm.bone_r(0.17), label="Prior")
# proxy2 = plt.Rectangle((10, 10), 1, 1, fc=plt.cm.Blues(0.57), label="Posterior")
ax[2,1].add_patch(proxy)
# ax[2,1].add_patch(proxy2)
ax[2,1].legend(loc="lower right")

bins_tmp = range(0.02, stop=0.165, length=50)
ax[1,1].hist([i[1] for i in mean_vals], bins=bins_tmp, color=plt.cm.OrRd(0.80), alpha=0.20)
ax[2,2].hist([i[2] for i in mean_vals], bins=bins_tmp, orientation="horizontal", color=plt.cm.OrRd(0.80), alpha=0.2)
ax[1,1].hist([i[1] for i in mean_vals_2], bins=bins_tmp, color=plt.cm.GnBu(0.80), alpha=0.20)
ax[2,2].hist([i[2] for i in mean_vals_2], bins=bins_tmp, orientation="horizontal", color=plt.cm.GnBu(0.80), alpha=0.20)

# cnt = ax[2,1].contourf(midpoints(hists[1].edges[1]), midpoints(hists[1].edges[2]), broadcast(x-> x>1 ? x : NaN, sum_weights), 
#     levels=range(minimum(sum_weights), maximum(sum_weights), length=11)[2:end], #corner_mask= true,
#     cmap="Blues", alpha=0.8, antialiased=true) 

# cnt = ax[2,1].contourf(midpoints(hists_2[1].edges[1]), midpoints(hists_2[1].edges[2]), broadcast(x-> x>1 ? x : NaN, sum_weights_2), 
#     levels=range(minimum(sum_weights_2), maximum(sum_weights_2), length=11)[2:end], #corner_mask= true,
#     cmap="Reds", alpha=0.8, antialiased=true) 

ax[1,1].set_ylim(bottom=0.0)
ax[2,2].set_xlim(left=0.0)

ax[1,2].set_visible(false)

ax[1,1].set_xticks([])
# ax[1,1].set_yticks([])
# ax[2,2].set_xticks([])
ax[2,2].set_yticks([])

ax[2,1].set_xlim(0.02, 0.165)
ax[1,1].set_xlim(0.02, 0.165)
ax[2,1].set_ylim(0.07, 0.165)
ax[2,2].set_ylim(0.07, 0.165)

ax[2,1].set_xlabel(L"\sigma_x \; (\mathrm{mm})")
ax[2,2].set_xlabel(L"\mathrm{\# Events}")
ax[2,1].set_ylabel(L"\sigma_y \; (\mathrm{mm})")
ax[1,1].set_ylabel(L"\mathrm{\# Events}")

## Plot 2: 

In [ ]:
hists = [hist_2d(i, 5,6) for i in fnames]
mean_vals = [marg_mean(i, 5,6) for i in fnames]
mean_vals_2 = [marg_mean(i, 7,8) for i in fnames]
std_vals = [marg_std(i, 5,6) for i in fnames]
std_vals_2 = [marg_std(i, 7,8) for i in fnames];

In [ ]:
xmin, xmax = 0, 10
xrange=range(xmin, stop = xmax,length=300)
yrange=range(xmin, stop = xmax,length=300)
xvals = [pdf(Normal(4, 2), x) for x in xrange]
yvals = [pdf(Normal(4, 2), y)  for y in yrange]
z = xvals*yvals';

In [ ]:
hists = [hist_2d(i, 5,6, axmin=[4,4], axmax=[8,8]) for i in fnames];
hists_2 = [hist_2d(i, 7,8, axmin=[1.0,1.0], axmax=[3.5,3.5]) for i in fnames];

In [ ]:
fig, ax= plt.subplots(2,2, figsize=(6,6), gridspec_kw=Dict("width_ratios"=>[6,1], "height_ratios"=>[1,6]))
fig.subplots_adjust(hspace=0.0, wspace=0.0)

# ax[2,1].contourf(xrange, yrange, z, levels=range(0.6*maximum(z), 1.4*maximum(z), length=50)[1:end], cmap="bone_r", alpha=0.7, antialiased=true)
range_tmp = range(0, stop=maximum(z), length=8)[2:end]
colors_tmp = [plt.cm.bone_r(i) for i in range(0, stop=0.4, length=length(range_tmp))[2:end]]
ax[2,1].contourf(xrange, yrange, z, levels=range_tmp, colors=colors_tmp, alpha=0.4, antialiased=true) 

sum_weights = zeros(size(hists[1].weights'))
sum_weights_2 = zeros(size(hists_2[1].weights'))

for i in eachindex(hists)
    sum_weights = sum_weights .+ hists[i].weights'
    sum_weights_2 = sum_weights_2 .+ hists_2[i].weights'
end

label_1 = "Data (Halo)"
label_2 = "Data (Core)"
for i in eachindex(hists)[end:-1:1]     
    ax[2,1].errorbar(mean_vals[i][1], mean_vals[i][2], xerr=std_vals[i][1], yerr=std_vals[i][2], 
        color="k", 
        fmt="o", markersize=4, 
        mfc=plt.cm.OrRd(charges_n[i]), #red
        markeredgecolor = "k", markeredgewidth=0.5,
        capsize=2, ecolor="k", elinewidth=0.5, label=label_1)
    label_1 = ""
    ax[2,1].errorbar(mean_vals_2[i][1], mean_vals_2[i][2], xerr=std_vals_2[i][1], yerr=std_vals_2[i][2], 
        color="k", 
        fmt="o", markersize=4, 
        mfc=plt.cm.GnBu(charges_n[i]), #red
        markeredgecolor = "k", markeredgewidth=0.5,
        capsize=2, ecolor="k", elinewidth=0.5, label=label_2)
    label_2 = ""
end

cb = ax[2,1].scatter(charges_n, charges_n, s=0.0, c=charges, cmap="OrRd")
cb2 = ax[2,1].scatter(charges_n, charges_n, s=0.0, c=charges, cmap="GnBu")
ax_ins_3 = fig.add_axes([0.19,0.55,0.02,0.2])
ax_ins_2 = fig.add_axes([0.15,0.55,0.02,0.2])
fig.colorbar(cb, spacing="proportional", label="Population", cax=ax_ins_3)
fig.colorbar(cb2, spacing="proportional", ticks=[], cax=ax_ins_2)

proxy = plt.Rectangle((10, 10), 1, 1, fc=plt.cm.bone_r(0.17), label="Prior")
# proxy2 = plt.Rectangle((10, 10), 1, 1, fc=plt.cm.Blues(0.57), label="Posterior LC")
# proxy3 = plt.Rectangle((10, 10), 1, 1, fc=plt.cm.Greens(0.57), label="Posterior SC")
ax[2,1].add_patch(proxy)
# ax[2,1].add_patch(proxy2)
# ax[2,1].add_patch(proxy3)
ax[2,1].legend(loc="lower right")

# bins_tmp = range(0.8, stop=7.2, length=50)
# ax[1,1].hist([i[1] for i in mean_vals], bins=bins_tmp, color=plt.cm.OrRd(0.80) , alpha=0.20)
# ax[1,1].hist([i[1] for i in mean_vals_2], bins=bins_tmp, color=plt.cm.GnBu(0.80), alpha=0.20)
# ax[2,2].hist([i[2] for i in mean_vals], bins=bins_tmp, orientation="horizontal", color=plt.cm.OrRd(0.80), alpha=0.20)
# ax[2,2].hist([i[2] for i in mean_vals_2], bins=bins_tmp, orientation="horizontal", color=plt.cm.GnBu(0.80), alpha=0.20)

# cnt = ax[2,1].contourf(midpoints(hists[1].edges[1]), midpoints(hists[1].edges[2]), broadcast(x-> x>1 ? x : NaN, sum_weights), 
#     levels=range(minimum(sum_weights), maximum(sum_weights), length=9)[2:end], #corner_mask= true,
#     cmap="Blues", alpha=0.6, antialiased=true) 

# cnt = ax[2,1].contourf(midpoints(hists_2[1].edges[1]), midpoints(hists_2[1].edges[2]), broadcast(x-> x>1 ? x : NaN, sum_weights_2), 
#     levels=range(minimum(sum_weights_2), maximum(sum_weights_2), length=9)[2:end], #corner_mask= true,
#     cmap="Reds", alpha=0.6, antialiased=true) 

# ax[1,1].set_ylim(bottom=0.0)
# ax[2,2].set_xlim(left=0.0)

# ax[1,2].set_visible(false)

# ax[1,1].set_xticks([])
# ax[2,2].set_yticks([])

# ax[2,1].set_xlim(0.8, 7.2)
# ax[1,1].set_xlim(0.8, 7.2)
# ax[2,1].set_ylim(0.8, 7.2)
# ax[2,2].set_ylim(0.8, 7.2)

# ax[2,1].set_xlabel(L"\sigma'_x \; (\mathrm{10^{-5} \cdot rad})")
# ax[2,2].set_xlabel(L"\mathrm{\# Events}")
# ax[2,1].set_ylabel(L"\sigma'_y \; (\mathrm{10^{-5} \cdot rad})")
# ax[1,1].set_ylabel(L"\mathrm{\# Events}")

## Plot #3

In [ ]:
hist1d_1 = [hist_1d(i, 11, nbins=50) for i in fnames]
mean_vals = [marg_mean(i, 11,11) for i in fnames];

In [ ]:
fig, ax= plt.subplots(1,1, figsize=(6,4), )

x_range = range(2.7, stop=3.3, length=100)
ax.fill_between(x_range, pdf.(Normal(2.9, 0.03), x_range), color=plt.cm.bone_r(0.3), label="Prior", alpha=0.5)

for i in eachindex(hists)  
    ax.plot(midpoints(hist1d_1[i].edges[1]), hist1d_1[i].weights, color=plt.cm.OrRd(charges_n[i]), alpha=0.7, lw=1)
end

cb = ax.scatter(charges_n, charges_n, s=0.0, c=charges, cmap="OrRd")
ax_ins_3 = fig.add_axes([0.15,0.67,0.02,0.2])
fig.colorbar(cb, spacing="proportional", label="Population", cax=ax_ins_3)

ax.axvline([2.9], color=plt.cm.bone_r(0.3), ls="--", label="Mean Prior")
ax.axvline(mean([i[2] for i in mean_vals]), color="red", ls="--", label="Mean Sampled")

ax.set_ylim(bottom=0.0,)
ax.set_xlim(2.75, 3.15)

ax.set_xlabel(L"\mathrm{Waist \; (m)}")
ax.set_ylabel("Counts")
ax.legend()

In [ ]:
xmin, xmax = 2.7, 3.3
xrange=range(xmin, stop = xmax,length=100)
yrange=range(1, stop = 30, length=100)
xvals = [pdf(Normal(2.9, 0.03), x) for x in xrange]
yvals = [1  for y in yrange]
z = xvals*yvals';

In [ ]:
mean_vals = [marg_mean(i, 10,11) for i in fnames];
std_vals = [marg_std(i, 10,11) for i in fnames];

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(6,6), )
fig.subplots_adjust(hspace=0.05, wspace=0.05)

ax[1].pcolormesh(xrange, yrange, z', cmap="bone_r", alpha=0.2, rasterized=true) 
ax[2].pcolormesh(xrange, yrange, z', cmap="bone_r", alpha=0.2, rasterized=true) 

label_1 = "Data (Halo)"
label_2 = "Data (Core)"

label_1 = "Data (Halo)"
label_2 = "Data (Core)"
for i in eachindex(mean_vals)   
#     ax[1].scatter(mean_vals[i][1], charges[i], color="darkred", s = 5, marker = "o", label=label_1)
#     ax[1].scatter(mean_vals[i][2], charges[i], marker = "s", s = 5, color="darkred", label=label_2)
#     ax[2].scatter(mean_vals[i][1], charges[i], marker = "o", s = 5, color="darkred",)
#     ax[2].scatter(mean_vals[i][2], charges[i], marker = "s", s = 5, color="darkred",)
    

    ax[1].errorbar(mean_vals[i][1], charges[i], xerr=std_vals[i][1],  
        color="k", 
        fmt="o", markersize=5, 
        mfc=plt.cm.GnBu(0.7), #red
        markeredgecolor = "k", markeredgewidth=0.5,
        capsize=2, ecolor="k", elinewidth=0.5, label=label_1)

    
    ax[1].errorbar(mean_vals[i][2], charges[i], xerr=std_vals[i][2], 
        color="k", 
        fmt="s", markersize=5, 
        mfc=plt.cm.OrRd(0.7), #red
        markeredgecolor = "k", markeredgewidth=0.5,
        capsize=2, ecolor="k", elinewidth=0.5, label=label_2)

    
    ax[2].errorbar(mean_vals[i][1], charges[i], xerr=std_vals[i][1],  
        color="k", 
        fmt="o", markersize=5, 
        mfc=plt.cm.GnBu(0.7), #red
        markeredgecolor = "k", markeredgewidth=0.5,
        capsize=2, ecolor="k", elinewidth=0.5, label=label_1)
    
    ax[2].errorbar(mean_vals[i][2], charges[i], xerr=std_vals[i][2], 
        color="k", 
        fmt="s", markersize=5, 
        mfc=plt.cm.OrRd(0.7), #red
        markeredgecolor = "k", markeredgewidth=0.5,
        capsize=2, ecolor="k", elinewidth=0.5, label=label_2)
    
    label_1 = ""
    label_2 = ""
end

ax[1].set_ylim(22,29)
ax[2].set_ylim(7,10.5)

ax[1].legend()

ax[2].set_xlabel(L"\mathrm{Waist \; (m)}")
fig.text(0.01, 0.5, "Population", va="center", rotation="vertical")

In [ ]:
hists = [hist_2d(i, 23,24, axmin=[2,2], axmax=[7,7]) for i in fnames];
mean_vals = [marg_mean(i, 23,24) for i in fnames];
# std_vals = [marg_std(i, 23,24) for i in fnames];

In [ ]:
xmin, xmax = 1, 7
xrange=range(xmin, stop = xmax,length=300)
yrange=range(xmin, stop = xmax,length=300)
xvals = [pdf(truncated(Normal(3, 1.5), 0, Inf), x) for x in xrange]
yvals = [pdf(truncated(Normal(3, 1.5), 0, Inf), y)  for y in yrange]
z = xvals*yvals';

In [ ]:
fig, ax= plt.subplots(1,1, figsize=(5,5),)

ax.contourf(xrange, yrange, z, levels=range(0.6*maximum(z), 1.4*maximum(z), length=50)[1:end], cmap="bone_r", alpha=0.7, antialiased=true) 

label = "Data Event"
for i in eachindex(hists)    
    ax.errorbar(mean_vals[i][1], mean_vals[i][2], #xerr=std_vals[i][1], yerr=std_vals[i][2], 
        color="k", 
        fmt="o", 
        mfc=plt.cm.OrRd(charges_n[i]), #red
        markeredgecolor = "k", markeredgewidth=0.5,
        capsize=2, ecolor="k", elinewidth=0.5, label=label)
    label = ""
end

cb = ax.scatter(charges_n, charges_n, s=0.0, c=charges, cmap="OrRd")
ax_ins_3 = fig.add_axes([0.15,0.65,0.02,0.2])
fig.colorbar(cb, spacing="proportional", label="Population", cax=ax_ins_3)

ax.set_xlabel("Res. x (px)")
ax.set_ylabel("Res. y (px)")

ax.set_xlim(1,7)
ax.set_ylim(1,7)

In [ ]:
function get_emmitance_halo(name, proj; α_min = 0.04, α_max = 0.96)
    # proj  1 / 2 
    smpl_tmp = bat_read(FPATH*name)
    smpl_tmp = varshape(prior).(smpl_tmp)
    smpl_tmp, smpl_w = flatview(unshaped.(smpl_tmp.v)), smpl_tmp.weight
    emm = smpl_tmp[proj,:].*smpl_tmp[4+proj,:].*10^-5 .* 1000 .* 426.0
    μ = mean(emm, FrequencyWeights(smpl_w))
    min =  quantile(emm, FrequencyWeights(smpl_w), α_min)
    max =  quantile(emm, FrequencyWeights(smpl_w), α_max)
    return (mean = μ, min = μ - min, max = max - μ)
end

function get_emmitance_core(name, proj; α_min = 0.04, α_max = 0.96)
    # proj  1 / 2 
    smpl_tmp = bat_read(FPATH*name)
    smpl_tmp = varshape(prior).(smpl_tmp)
    smpl_tmp, smpl_w = flatview(unshaped.(smpl_tmp.v)), smpl_tmp.weight
    emm = smpl_tmp[2+proj,:].*smpl_tmp[6+proj,:].*10^-5 .* 1000 .* 426.0
    μ = mean(emm, FrequencyWeights(smpl_w))
    min =  quantile(emm, FrequencyWeights(smpl_w), α_min)
    max =  quantile(emm, FrequencyWeights(smpl_w), α_max)
    return (mean = μ, min = μ - min, max = max - μ)
end

In [ ]:
emm_tmp_1 = [get_emmitance_halo(i, 1) for i in fnames]
emm_tmp_2 = [get_emmitance_halo(i, 2) for i in fnames];

In [ ]:
emm_tmp_21 = [get_emmitance_core(i, 1) for i in fnames]
emm_tmp_22 = [get_emmitance_core(i, 2) for i in fnames];

In [ ]:
prior_emmitance_x = rand(Normal(0.2, 0.04), 2*10^6) .* rand(Normal(4, 2), 2*10^6) .* 10^-5 .* 1000 .* 426.0
prior_emmitance_y = rand(Normal(0.2, 0.04), 2*10^6) .* rand(Normal(4, 2), 2*10^6) .* 10^-5 .* 1000 .* 426.0

hist_em_tmp = fit(Histogram, (prior_emmitance_x,prior_emmitance_y),nbins=50)
hist_em_tmp = normalize(hist_em_tmp, mode=:pdf);

range_tmp = range(minimum(hist_em_tmp.weights), stop=maximum(hist_em_tmp.weights), length=7)[2:end]
colors_tmp = [plt.cm.bone_r(i) for i in range(0, stop=0.5, length=length(range_tmp))[2:end]];

In [ ]:
fig, ax= plt.subplots(2,2, figsize=(6,6), gridspec_kw=Dict("width_ratios"=>[6,1], "height_ratios"=>[1,6]))
fig.subplots_adjust(hspace=0.0, wspace=0.0)

ax[2,1].contourf(midpoints(hist_em_tmp.edges[1]), midpoints(hist_em_tmp.edges[2]), hist_em_tmp.weights', levels=range_tmp, colors=colors_tmp, alpha=0.3, antialiased=true) 

label_1 = "Data (Halo)"
label_2 = "Data (Core)"

for i in eachindex(emm_tmp_1)[end:-1:1]
    
    ax[2,1].errorbar(emm_tmp_1[i].mean, emm_tmp_2[i].mean, 
        xerr=[[emm_tmp_1[i].min], [emm_tmp_1[i].max]], 
        yerr=[[emm_tmp_2[i].min], [emm_tmp_2[i].max]], 
        color="k", fmt="o", markersize = 4.5,
        mfc=plt.cm.OrRd(charges_n[i]),
        markeredgecolor = "k", markeredgewidth=0.4, 
        capsize=1.2, ecolor="k", elinewidth=0.4, errorevery=10, label=label_1
        )

    ax[2,1].errorbar(emm_tmp_21[i].mean, emm_tmp_22[i].mean, 
        xerr=[[emm_tmp_21[i].min], [emm_tmp_21[i].max]], 
        yerr=[[emm_tmp_22[i].min], [emm_tmp_22[i].max]], 
        color="k", fmt="o", markersize = 4.5,
        mfc=plt.cm.GnBu(charges_n[i]),
        markeredgecolor = "k", markeredgewidth=0.4, 
        capsize=1.2, ecolor="k", elinewidth=0.4, errorevery=10, label=label_2
        )
    label_2 = ""
    label_1 = ""
end

proxy = plt.Rectangle((10, 10), 1, 1, fc=plt.cm.bone_r(0.17), label="Prior")
ax[2,1].add_patch(proxy)

ax[2,1].legend(loc="lower right")

cb = ax[2,1].scatter(charges_n, charges_n, s=0.0, c=charges, cmap="OrRd")
cb2 = ax[2,1].scatter(charges_n, charges_n, s=0.0, c=charges, cmap="GnBu")
ax_ins_3 = fig.add_axes([0.67,0.25,0.02,0.2])
ax_ins_4 = fig.add_axes([0.63,0.25,0.02,0.2])

fig.colorbar(cb, spacing="proportional", label="Population", cax=ax_ins_3)
fig.colorbar(cb2, spacing="proportional", ticks=[], cax=ax_ins_4)

bins_tmp = range(0.7, stop=3.5, length=50)
ax[1,1].hist([i.mean for i in emm_tmp_21], bins=bins_tmp, color=plt.cm.GnBu(0.80), alpha=0.20)
ax[2,2].hist([i.mean for i in emm_tmp_22], bins=bins_tmp, orientation="horizontal", color=plt.cm.GnBu(0.80), alpha=0.20)
ax[1,1].hist([i.mean for i in emm_tmp_1], bins=bins_tmp, color=plt.cm.OrRd(0.80), alpha=0.20)
ax[2,2].hist([i.mean for i in emm_tmp_2], bins=bins_tmp, orientation="horizontal", color=plt.cm.OrRd(0.80), alpha=0.20)

ax[1,1].set_ylim(bottom=0.0)
ax[2,2].set_xlim(left=0.0)

ax[1,2].set_visible(false)

ax[1,1].set_xticks([])
ax[2,2].set_yticks([])

ax[2,1].set_xlim(0.7, 3.5)
ax[1,1].set_xlim(0.7, 3.5)
ax[2,1].set_ylim(0.7, 3.5)
ax[2,2].set_ylim(0.7, 3.5)

ax[2,1].set_xlabel(L"\epsilon_x \; (\mathrm{mm \cdot mrad})")
ax[2,2].set_xlabel(L"\mathrm{\# Events}")
ax[2,1].set_ylabel(L"\epsilon_y \; (\mathrm{mm \cdot mrad})")
ax[1,1].set_ylabel(L"\mathrm{\# Events}")

In [ ]:
mean_vals1 = [marg_mean(i, 12,16) for i in fnames]
mean_vals2 = [marg_mean(i, 13,17) for i in fnames]
mean_vals3 = [marg_mean(i, 14,18) for i in fnames]
mean_vals4 = [marg_mean(i, 15,19) for i in fnames]

std_vals1 = [marg_std(i, 12,16) for i in fnames]
std_vals2 = [marg_std(i, 13,17) for i in fnames]
std_vals3 = [marg_std(i, 14,18) for i in fnames]
std_vals4 = [marg_std(i, 15,19) for i in fnames];

In [ ]:
param_tmp = rand(prior)

μ1x = [i[1]*param_tmp.psx[1] for i in mean_vals1]; μ1x = μ1x .- minimum(μ1x)
μ1y = [i[2]*param_tmp.psy[1] for i in mean_vals1]; μ1y = μ1y .- minimum(μ1y)

μ2x = [i[1]*param_tmp.psx[2] for i in mean_vals2]; μ2x = μ2x .- minimum(μ2x)
μ2y = [i[2]*param_tmp.psy[2] for i in mean_vals2]; μ2y = μ2y .- minimum(μ2y)

μ3x = [i[1]*param_tmp.psx[3] for i in mean_vals3]; μ3x = μ3x .- minimum(μ3x)
μ3y = [i[2]*param_tmp.psy[3] for i in mean_vals3]; μ3y = μ3y .- minimum(μ3y)

μ4x = [i[1]*param_tmp.cam4_psx for i in mean_vals4]; μ4x = μ4x .- minimum(μ4x)
μ4y = [i[2]*param_tmp.cam4_psy for i in mean_vals4]; μ4y = μ4y .- minimum(μ4y);

In [ ]:
fig, ax= plt.subplots(2,2, figsize=(6,6),)
fig.subplots_adjust(hspace=0.07, wspace=0.07)

label = "Mean + Std"

ax[1,1].scatter(μ1x, μ1y, c=plt.cm.OrRd(runind_n), marker="o", s = 20, edgecolors="k", linewidths=0.5, label=label)
ax[1,2].scatter(μ2x, μ2y, c=plt.cm.OrRd(runind_n), marker="o", s = 20, edgecolors="k", linewidths=0.5, )
ax[2,1].scatter(μ3x, μ3y, c=plt.cm.OrRd(runind_n), marker="o", s = 20, edgecolors="k", linewidths=0.5, )
ax[2,2].scatter(μ4x, μ4y, c=plt.cm.OrRd(runind_n), marker="o", s = 20, edgecolors="k", linewidths=0.5, )
    
    
# for i in eachindex(hists)    
#     ax[1,1].scatter(μ1x, μ1y,  
#          color=plt.cm.OrRd(runind_n[i])
# #         fmt="o", markersize = 5,
# #         mfc=plt.cm.OrRd(runind_n[i]), #red
# #         markeredgecolor = "k", markeredgewidth=0.5, 
# #         capsize=1.5, ecolor="k", elinewidth=0.5, label=label
#     )
#     label = ""
# #     ax[1,2].errorbar(mean_vals2[i][1], mean_vals2[i][2], xerr=std_vals2[i][1], yerr=std_vals2[i][2], 
# #         color="k", 
# #         fmt="o", markersize = 5,
# #         mfc=plt.cm.OrRd(runind_n[i]), #red
# #         markeredgecolor = "k", markeredgewidth=0.5, 
# #         capsize=1.5, ecolor="k", elinewidth=0.5, label=label)
# #     ax[2,1].errorbar(mean_vals3[i][1], mean_vals3[i][2], xerr=std_vals3[i][1], yerr=std_vals3[i][2], 
# #         color="k", 
# #         fmt="o", markersize = 5,
# #         mfc=plt.cm.OrRd(runind_n[i]), #red
# #         markeredgecolor = "k", markeredgewidth=0.5, 
# #         capsize=1.5, ecolor="k", elinewidth=0.5, label=label)
# #     ax[2,2].errorbar(mean_vals4[i][1], mean_vals4[i][2], xerr=std_vals4[i][1], yerr=std_vals4[i][2], 
# #         color="k", 
# #         fmt="o", markersize = 5,
# #         mfc=plt.cm.OrRd(runind_n[i]), #red
# #         markeredgecolor = "k", markeredgewidth=0.5, 
# #         capsize=1.5, ecolor="k", elinewidth=0.5, label=label)
# end

cb = ax[2,2].scatter(charges_n, charges_n, s=0.0, c=runind, cmap="OrRd")
ax_ins_3 = fig.add_axes([0.79,0.37,0.01,0.1])
fig.colorbar(cb, spacing="proportional", label="Event #", cax=ax_ins_3)

ax[1,1].xaxis.tick_top()
ax[1,1].xaxis.set_ticks_position("top")

ax[1,2].xaxis.tick_top()
ax[1,2].xaxis.set_ticks_position("top")
ax[1,2].yaxis.tick_right()
ax[1,2].yaxis.set_ticks_position("right")

ax[2,2].yaxis.tick_right()
ax[2,2].yaxis.set_ticks_position("right")

ax[1,1].text(0.04,0.91,"Cam. #1", transform=ax[1,1].transAxes)
ax[1,2].text(0.04,0.91,"Cam. #2", transform=ax[1,2].transAxes)
ax[2,1].text(0.04,0.91,"Cam. #3", transform=ax[2,1].transAxes)
ax[2,2].text(0.04,0.91,"Cam. #4", transform=ax[2,2].transAxes)

ax[1,1].set_xlim(-5,180)
ax[1,1].set_ylim(-5,180)

ax[1,2].set_xlim(-5,180)
ax[1,2].set_ylim(-5,180)

ax[2,1].set_xlim(-5,200)
ax[2,1].set_ylim(-5,200)

ax[2,2].set_xlim(-5,300)
ax[2,2].set_ylim(-5,300)
# ax[2,2].set_ylim(34.2,37)

ax[2,2].set_xlabel(L"\mu_x \; \mathrm{(\mu m)}")
ax[2,1].set_xlabel(L"\mu_x \; \mathrm{(\mu m)}")

ax[1,1].set_ylabel(L"\mu_y \; \mathrm{(\mu m)}")
ax[2,1].set_ylabel(L"\mu_y \; \mathrm{(\mu m)}")